# Main Script

STATIC VERSION
- Given a fixed DoD
- Main function
- Auxillary functions:
    - initial solution creator: fill up vehicles while respecting capacity
    - (greedy) insertion heuristic: in stead of looking for all possible insertions, look for a better insertion until you find no better one
        - initial solution creator > spatio-temporal clustering of requests?
        - feasibility check
        - incremental waiting time + travel time calculator
    - improvement heuristic
        - neighborhood creators (destroy&repair, swappers, L-opt moves, ...)
        - evaluate objective functions

## Todo (update 16/2)

- Test basic network loading functions for larger instances! 
- Some vehicles can drive all the way to 5 (1 > 5 > 1), others return from 3 directly back (1 > 3 > 1)
    - make it that some vehicles are available at the depot: these can be schedules for new rides

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

import network_generation as netg
import requests_generation as rg
import solution_generation as sg
import solution_evaluation as se
import solution_visualisation as sv

In [2]:
# PARAMETERS

#network
network_size = 'small'
interstop_distance = 'half'
v_mean = 50 #km/h

#requests
demand_scenario = 2
time_of_day = 1 #1 = peak, 0 = off-peak
peak_duration = 60 #min.
req_max_cluster_time = 10 #min.

#vehicles
available_vehicles = 35
vehicle_capacity = 20
dod = 0   #static version = 0, dynamic version > 0

#objective function
WT_pen = 3 
TT_pen = 1 

## Load network

In [3]:
# network
network = netg.import_network(network_size, interstop_distance)
od_matrix = netg.generate_cost_matrix(network, v_mean)
od_matrix

{(1, 1): 0.0,
 (1, 2): 5.9687184554140265,
 (1, 3): 6.0,
 (1, 4): 5.9687184554140265,
 (1, 5): 0.0,
 (2, 1): 5.9687184554140265,
 (2, 2): 0.0,
 (2, 3): 5.9687184554140265,
 (2, 4): 0.0,
 (2, 5): 5.9687184554140265,
 (3, 1): 6.0,
 (3, 2): 5.9687184554140265,
 (3, 3): 0.0,
 (3, 4): 5.9687184554140265,
 (3, 5): 6.0,
 (4, 1): 5.9687184554140265,
 (4, 2): 0.0,
 (4, 3): 5.9687184554140265,
 (4, 4): 0.0,
 (4, 5): 5.9687184554140265,
 (5, 1): 0.0,
 (5, 2): 5.9687184554140265,
 (5, 3): 6.0,
 (5, 4): 5.9687184554140265,
 (5, 5): 0.0}

## Import requests

In [4]:
lambdapeak = rg.get_scenario_mean_demand('city', network_size, scen=demand_scenario, peak=1)
mupeak = rg.get_scenario_mean_demand('terminal', network_size, scen=demand_scenario, peak=1)

In [5]:
dict_requests = rg.convert_md_todict(lambdapeak, mupeak, demand_scenario)
total_requests = rg.generate_requests(dict_requests, peak_duration)
od_pairs = set(total_requests.keys())

In [6]:
list_all_requests = rg.list_all_requests(total_requests)
list_all_requests
len(list_all_requests)

626

In [7]:
# requests are grouped per 5 minutes from the 1st departure of a group
copy_list_requests = list_all_requests.copy()
grouped_requests = rg.group_requests_dt(copy_list_requests, req_max_cluster_time, od_pairs)
grouped_requests

{(2,
  3): [[((2, 3), 0, 0),
   ((2, 3), 0.2, 0),
   ((2, 3), 0.4, 0),
   ((2, 3), 0.6, 0),
   ((2, 3), 0.8, 0),
   ((2, 3), 1.0, 0),
   ((2, 3), 1.2, 0),
   ((2, 3), 1.4, 0),
   ((2, 3), 1.6, 0),
   ((2, 3), 1.8, 0),
   ((2, 3), 2.0, 0),
   ((2, 3), 2.2, 0),
   ((2, 3), 2.4, 0),
   ((2, 3), 2.6, 0),
   ((2, 3), 2.8, 0),
   ((2, 3), 3.0, 0),
   ((2, 3), 3.2, 0),
   ((2, 3), 3.4, 0),
   ((2, 3), 3.6, 0),
   ((2, 3), 3.8, 0),
   ((2, 3), 4.0, 0),
   ((2, 3), 4.2, 0),
   ((2, 3), 4.4, 0),
   ((2, 3), 4.6, 0),
   ((2, 3), 4.8, 0),
   ((2, 3), 5.0, 0),
   ((2, 3), 5.2, 0),
   ((2, 3), 5.4, 0),
   ((2, 3), 5.6, 0),
   ((2, 3), 5.8, 0),
   ((2, 3), 6.0, 0),
   ((2, 3), 6.2, 0),
   ((2, 3), 6.4, 0),
   ((2, 3), 6.6, 0),
   ((2, 3), 6.8, 0),
   ((2, 3), 7.0, 0),
   ((2, 3), 7.2, 0),
   ((2, 3), 7.4, 0),
   ((2, 3), 7.6, 0),
   ((2, 3), 7.8, 0),
   ((2, 3), 8.0, 0),
   ((2, 3), 8.2, 0),
   ((2, 3), 8.4, 0),
   ((2, 3), 8.6, 0),
   ((2, 3), 8.8, 0),
   ((2, 3), 9.0, 0),
   ((2, 3), 9.2, 0),
   ((

In [8]:
# this shows how big the groups of passengers are (per threshold time from the first departure) + how many there are
count_groups = rg.request_groups_per_od(grouped_requests)
count_groups

{(2, 3): [51, 51, 51, 51, 51, 45],
 (1, 3): [51, 51, 51, 51, 51, 45],
 (4, 5): [1, 1],
 (1, 2): [2, 2, 2, 2],
 (3, 5): [2, 2, 2, 2],
 (3, 4): [2, 2, 2, 2]}

## Initial solution generation

Insertion strategy: momentarily focussed on the capacity constraint (i.e. trying to fill vehicles), not so much on the time constraint

1. Divide requests into chunks: 
 * (a) of at most equal size (e.g. 10) 
 * (b) group requests as long as the maximum waiting time is e.g. 5 min.
2. Select a first chunk of travellers, who want to travel from the origin to the farthest point, and fill a bus with them (almost) 
    * If this bus is ride is full, then this schedule is complete.
    * If there is room left, then add passengers in between at each visited stop.


In [9]:
# MOVE this to the parameter section!
terminal, city, terminal_end = netg.get_network_boundaries(network)
network_dim = terminal, city, terminal_end
# find a more systematic way to decide which services make the entire trip
round_trips = {1, 2, 3, 4}

first_veh_index = 1

copy_grouped_requests = grouped_requests.copy()

In [10]:
initial = sg.create_initial_solution(copy_grouped_requests, city, terminal_end, network_dim, first_veh_index,  
                                     available_vehicles, round_trips, vehicle_capacity)

In [11]:
# check if all requests are indeed added ot a schedule
copy_grouped_requests

{(2, 3): [[], [], [], [], [], []],
 (1, 3): [[], [], [], [], [], []],
 (4, 5): [[], []],
 (1, 2): [[], [], [], []],
 (3, 5): [[], [], [], []],
 (3, 4): [[], [], [], []]}

### Correction to schedules

In [12]:
# Correct for the departure times
corr_initial = sg.correct_dep_times(initial, od_matrix, round_trips, network_dim)

In [13]:
# Merge services into vehicle schedules
vehicle_schedules = sg.services_to_vehicles(corr_initial, round_trips, network_dim, od_matrix, 3)
vehicle_schedules

{(1,
  1): {1: [7.96,
   [((1, 3), 0, 0),
    ((1, 3), 0.2, 0),
    ((1, 3), 0.4, 0),
    ((1, 3), 0.6, 0),
    ((1, 3), 0.8, 0),
    ((1, 3), 1.0, 0),
    ((1, 3), 1.2, 0),
    ((1, 3), 1.4, 0),
    ((1, 3), 1.6, 0),
    ((1, 3), 1.8, 0)],
   [((1, 2), 0, 0), ((1, 2), 7.96, 0)]], 2: [13.93,
   [((1, 3), 0, 0),
    ((1, 3), 0.2, 0),
    ((1, 3), 0.4, 0),
    ((1, 3), 0.6, 0),
    ((1, 3), 0.8, 0),
    ((1, 3), 1.0, 0),
    ((1, 3), 1.2, 0),
    ((1, 3), 1.4, 0),
    ((1, 3), 1.6, 0),
    ((1, 3), 1.8, 0)],
   [((2, 3), 0, 0),
    ((2, 3), 0.2, 0),
    ((2, 3), 0.4, 0),
    ((2, 3), 0.6, 0),
    ((2, 3), 0.8, 0),
    ((2, 3), 1.0, 0),
    ((2, 3), 1.2, 0),
    ((2, 3), 1.4, 0),
    ((2, 3), 1.6, 0),
    ((2, 3), 1.8, 0)]], 3: [19.9,
   [((3, 5), 0, 0), ((3, 5), 7.96, 0)],
   [((3, 4), 0, 0), ((3, 4), 7.96, 0)]], 4: [25.87,
   [((3, 5), 0, 0), ((3, 5), 7.96, 0)],
   [((4, 5), 0, 0)]], 5: [31.84]},
 (1,
  2): {1: [39.8,
   [((1, 3), 4.0, 0),
    ((1, 3), 4.2, 0),
    ((1, 3), 4.4, 0),
   

## Evaluate solution

In [14]:
waiting_time_dict = se.calc_waiting_time(vehicle_schedules)
inveh_time_dict = se.calc_in_vehicle_time(vehicle_schedules)
total_tt_dict = se.calculate_ttt(inveh_time_dict, waiting_time_dict)

In [15]:
# check if indeed all requests are added to a schedule
copy_grouped_requests

{(2, 3): [[], [], [], [], [], []],
 (1, 3): [[], [], [], [], [], []],
 (4, 5): [[], []],
 (1, 2): [[], [], [], []],
 (3, 5): [[], [], [], []],
 (3, 4): [[], [], [], []]}

In [16]:
sum_stops = se.sum_total_tt(total_tt_dict, level='stop')
sum_vehicle = se.sum_total_tt(total_tt_dict, level='vehicle')
sum_total = se.sum_total_tt(total_tt_dict, level='total')

# sum splitted into respecitvely waiting & iv time
wt_stops = se.sum_total_tt(waiting_time_dict, level='stop')
ivt_stops = se.sum_total_tt(inveh_time_dict, level='stop')

In [17]:
sum_stops

{(1, 1): {1: 209.89999999999998,
  2: 189.99999999999997,
  3: 99.5,
  4: 31.84,
  5: 0},
 (1, 2): {1: 488.30000000000007, 2: 468.40000000000003, 3: 99.5, 4: 0, 5: 0},
 (1, 3): {1: 627.5, 2: 607.6, 3: 99.49999999999999, 4: 0, 5: 0},
 (2, 1): {1: 349.09999999999997,
  2: 329.19999999999993,
  3: 123.34,
  4: 5.969999999999999,
  5: 0},
 (5, 1): {1: 128.39999999999998, 2: 128.39999999999998, 3: 0, 4: 0, 5: 0},
 (6, 1): {1: 11.940000000000001, 2: 192.66000000000003, 3: 0, 4: 0, 5: 0},
 (7, 1): {1: 128.4, 2: 92.40000000000002, 3: 0, 4: 0, 5: 0},
 (8, 1): {1: 128.4, 2: 92.40000000000002, 3: 0, 4: 0, 5: 0},
 (9, 1): {1: 128.4, 2: 92.40000000000002, 3: 0, 4: 0, 5: 0},
 (10, 1): {1: 128.4, 2: 92.40000000000002, 3: 0, 4: 0, 5: 0},
 (11, 1): {1: 128.4, 2: 92.40000000000002, 3: 0, 4: 0, 5: 0},
 (12, 1): {1: 13.169999999999998, 2: 147.62999999999994, 3: 0, 4: 0, 5: 0},
 (13, 1): {1: 140.69999999999996, 2: 68.69999999999997, 3: 0, 4: 0, 5: 0},
 (14, 1): {1: 140.69999999999996, 2: 68.69999999999997,

In [18]:
sum_vehicle

{(1, 1): 531.24,
 (1, 2): 1056.2,
 (1, 3): 1334.6,
 (2, 1): 807.61,
 (5, 1): 256.79999999999995,
 (6, 1): 204.60000000000002,
 (7, 1): 220.8,
 (8, 1): 220.8,
 (9, 1): 220.8,
 (10, 1): 220.8,
 (11, 1): 220.8,
 (12, 1): 160.79999999999993,
 (13, 1): 209.39999999999992,
 (14, 1): 209.39999999999992,
 (15, 1): 209.39999999999992,
 (16, 1): 209.39999999999992,
 (17, 1): 209.39999999999992,
 (18, 1): 164.40000000000003,
 (19, 1): 245.4,
 (20, 1): 245.40000000000006,
 (21, 1): 245.40000000000006,
 (22, 1): 245.40000000000006,
 (23, 1): 245.40000000000006,
 (24, 1): 167.99999999999997,
 (25, 1): 281.4,
 (26, 1): 281.4,
 (27, 1): 225.77999999999994,
 (28, 1): 128.39999999999998,
 (29, 1): 128.39999999999998,
 (30, 1): 11.940000000000005,
 (31, 1): 128.39999999999995,
 (32, 1): 128.39999999999995,
 (33, 1): 128.39999999999995,
 (34, 1): 128.39999999999995}

In [19]:
sum_total

9633.169999999996

In [20]:
occ = se.calc_occupancy_rate(vehicle_schedules, vehicle_capacity)

## Solution visualisation

In [24]:
df_solution = sv.convert_to_dataframe(vehicle_schedules)
df_wt = sv.convert_to_dataframe(wt_stops)
df_ivt = sv.convert_to_dataframe(ivt_stops)
df_occ = sv.convert_to_dataframe(occ)

col_names = ['dep_time','abboard_pax1','abboard_pax2','abboard_pax3','sum_wt','sum_ivt','veh_occ']
df_all = pd.concat([df_solution,df_wt, df_ivt, df_occ], axis=1)
df_all.columns = col_names

In [25]:
df_all

,dep_time,abboard_pax1,abboard_pax2,abboard_pax3,sum_wt,sum_ivt,veh_occ
"((1, 1), 1)",7.96,"[((1, 3), 0, 0), ((1, 3), 0.2, 0), ((1, 3), 0....","[((1, 2), 0, 0), ((1, 2), 7.96, 0)]",None,78.56,131.34,0.60
"((1, 1), 2)",13.93,"[((1, 3), 0, 0), ((1, 3), 0.2, 0), ((1, 3), 0....","[((2, 3), 0, 0), ((2, 3), 0.2, 0), ((2, 3), 0....",None,130.30,59.70,1.00
"((1, 1), 3)",19.90,"[((3, 5), 0, 0), ((3, 5), 7.96, 0)]","[((3, 4), 0, 0), ((3, 4), 7.96, 0)]",None,63.68,35.82,0.20
"((1, 1), 4)",25.87,"[((3, 5), 0, 0), ((3, 5), 7.96, 0)]","[((4, 5), 0, 0)]",None,25.87,5.97,0.15
"((1, 1), 5)",31.84,None,None,None,0.00,0.00,0.00
...,...,...,...,...,...,...,...
"((34, 1), 1)",58.80,"[((1, 3), 57.0, 0), ((1, 3), 57.2, 0), ((1, 3)...",None,None,9.00,119.40,0.50
"((34, 1), 2)",64.77,"[((1, 3), 57.0, 0), ((1, 3), 57.2, 0), ((1, 3)...",None,None,0.00,0.00,0.50
"((34, 1), 3)",70.74,None,None,None,0.00,0.00,0.00
"((34, 1), 4)",NaN,None,None,None,0.00,0.00,0.00


In [26]:
df_all.to_excel("Exports/entire_solution.xlsx")